# Modelo de detecção de plágio

Agora que você criou dados de treinamento e teste, está pronto para definir e treinar um modelo. Seu objetivo neste bloco de notas será treinar um modelo de classificação binária que aprenda a rotular um arquivo de resposta como plagiado ou não, com base nos recursos que você fornece ao modelo.

Esta tarefa será dividida em algumas etapas discretas:

* Envie seus dados para o S3.
* Defina um modelo de classificação binária e um script de treinamento.
* Treine seu modelo e implante-o.
* Avalie seu classificador implantado e responda a algumas perguntas sobre sua abordagem.

Para completar este caderno, você terá que completar todos os exercícios dados e responder a todas as perguntas neste caderno.
> Todas as suas tarefas serão claramente rotuladas como **EXERCÍCIO** e as perguntas como **QUESTION**.

Caberá a você explorar diferentes modelos de classificação e decidir sobre um modelo que ofereça o melhor desempenho para este conjunto de dados.

---

## Carregar dados para S3

No último bloco de notas, você deve ter criado dois arquivos: um arquivo `training.csv` e` test.csv` com os recursos e rótulos de classe para o corpus dado de dados de texto plagiado / não plagiado.

> As células abaixo são carregadas em algumas bibliotecas do AWS SageMaker e cria um intervalo padrão. Depois de criar esse bucket, você pode carregar seus dados armazenados localmente para o S3.

Salve seus arquivos treino e teste de recurso `.csv`, localmente. Para fazer isso, você pode executar o segundo bloco de notas "2_Plagiarism_Feature_Engineering" no SageMaker ou pode carregar manualmente seus arquivos para este bloco de notas usando o ícone de upload no Laboratório Jupyter. Então você pode fazer upload de arquivos locais para o S3 usando `sagemaker_session.upload_data` e apontando diretamente para onde os dados de treinamento são salvos.

In [1]:
import numpy as np
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCÍCIO: Envie seus dados de treinamento para S3

Especifique o `data_dir` onde você salvou seu arquivo` train.csv`. Escolha um `prefixo` descritivo que define onde seus dados serão carregados no bucket S3 padrão. Finalmente, crie um ponteiro para seus dados de treinamento chamando `sagemaker_session.upload_data` e passando os parâmetros necessários. Pode ser útil olhar a [documentação da sessão](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) ou exemplos de código SageMaker anteriores.

Espera-se que você carregue todo o seu diretório. Posteriormente, o script de treinamento acessará apenas o arquivo `train.csv`.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism-detection'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_data
#test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
#train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

's3://sagemaker-sa-east-1-377572488885/plagiarism-detection'

### Célula de teste

Teste se seus dados foram carregados com sucesso. A célula abaixo imprime os itens em seu balde S3 e gerará um erro se estiver vazio. Você deve ver o conteúdo do seu `data_dir` e talvez alguns pontos de verificação. Se você vir qualquer outro arquivo listado, então você pode ter alguns arquivos de modelo antigo que você pode excluir através do console S3 (entretanto, arquivos adicionais não devem afetar o desempenho do modelo desenvolvido neste notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism-detection/test.csv
plagiarism-detection/train.csv
Test passed!


---

# Modelagem

Agora que você carregou seus dados de treinamento, é hora de definir e treinar um modelo!

O tipo de modelo que você cria é com você. Para uma tarefa de classificação binária, você pode escolher seguir um dos três caminhos:
* Use um algoritmo de classificação integrado, como LinearLearner.
* Defina um classificador Scikit-learn personalizado, uma comparação de modelos pode ser encontrada [aqui](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Defina um classificador de rede neural PyTorch personalizado.

Caberá a você testar uma variedade de modelos e escolher o melhor. Seu projeto será avaliado com base na precisão de seu modelo final.
 
---

## EXERCÍCIO: Complete um script de treinamento

Para implementar um classificador personalizado, você precisará completar um script `train.py`. Você recebeu as pastas `source_sklearn` e` source_pytorch` que contêm o código inicial para um modelo Scikit-learn personalizado e um modelo PyTorch, respectivamente. Cada diretório possui um script de treinamento `train.py`. Para concluir este projeto **você só precisa concluir um desses scripts**; o script que é responsável por treinar seu modelo final.

Um script de treinamento típico:
* Carrega dados de treinamento de um diretório especificado
* Analisa qualquer treinamento e hiperparâmetros de modelo (por exemplo, nós em uma rede neural, períodos de treinamento, etc.)
* Instancia um modelo do seu design, com quaisquer hiperparâmetros especificados
* Treina esse modelo
* Por fim, salva o modelo para que possa ser hospedado / implantado posteriormente

### Definindo e treinando um modelo
Muito do código do script de treinamento é fornecido para você. Quase todo o seu trabalho será feito na seção ʻif __name__ == '__main __': `. Para completar um arquivo `train.py`, você irá:
1. Importe todas as bibliotecas extras de que precisar
2. Defina quaisquer hiperparâmetros adicionais de treinamento de modelo usando `parser.add_argument`
2. Defina um modelo na seção ʻif __name__ == '__main __': `
3. Treine o modelo na mesma seção

Abaixo, você pode usar `! Pygementize` para exibir um arquivo` train.py` existente. Leia o código; todas as suas tarefas são marcadas com comentários `TODO`.

**Nota: Se você escolher criar um modelo PyTorch personalizado, você será responsável por definir o modelo no arquivo `model.py`, ** e um arquivo` predict.py` é fornecido. Se você escolher usar o Scikit-learn, você só precisa do arquivo `train.py`; você pode importar um classificador da biblioteca `sklearn`.

### Código fornecido

Se você leu o código acima, pode ver que o código inicial inclui algumas coisas:
* Carregamento do modelo (`model_fn`) e salvamento do código
* Obtendo os hiperparâmetros padrão do SageMaker
* Carregando os dados de treinamento por nome, `train.csv` e extraindo os recursos e rótulos,` train_x` e `train_y`

Se você gostaria de ler mais sobre como salvar modelo com [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) ou com [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), clique nos links fornecidos.

---
# Crie um estimador

Quando um modelo personalizado é construído no SageMaker, um ponto de entrada deve ser especificado. Este é o arquivo Python que será executado quando o modelo for treinado; a função `train.py` que você especificou acima. Para executar um script de treinamento personalizado no SageMaker, construa um estimador e preencha os argumentos do construtor apropriados:

* **entry_point**: o caminho para o script Python que SageMaker executa para treinamento e previsão.
* **source_dir**: O caminho para o diretório do script de treinamento `source_sklearn` OU` source_pytorch`.
* **entry_point**: o caminho para o script Python que SageMaker executa para treinamento e previsão.
* **source_dir**: O caminho para o diretório do script de treinamento `train_sklearn` OU` train_pytorch`.
* **entry_point**: O caminho para o script Python que SageMaker executa para treinamento.
* **source_dir**: O caminho para o diretório do script de treinamento `train_sklearn` OU` train_pytorch`.
* **função**: Função ARN, que foi especificada acima.
* **train_instance_count**: O número de instâncias de treinamento (deve ser deixado em 1).
* **train_instance_type**: O tipo de instância SageMaker para treinamento. Observação: como o Scikit-learn não oferece suporte nativo ao treinamento de GPU, o Sagemaker Scikit-learn não oferece suporte a treinamento em tipos de instância de GPU.
* **sagemaker_session**: A sessão usada para treinar no Sagemaker.
* **hiperparâmetros** (opcional): Um dicionário `{'nome': valor, ..}` passado para a função de trem como hiperparâmetros.

Nota: Para um modelo PyTorch, há outro argumento opcional **framework_version**, que você pode definir para a versão mais recente do PyTorch, `1.0`.

## EXERCÍCIO: Definir um estimador Scikit-learn ou PyTorch

Para importar o estimador desejado, use uma das seguintes linhas:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [6]:
# your import and estimator code, here

# from sagemaker.sklearn.estimator import SKLearn
# from sagemaker.pytorch import PyTorch

from sagemaker import LinearLearner

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [7]:
output_path = 's3://{}/{}'.format(bucket, prefix)

In [8]:
linear = LinearLearner(role = role,
                      train_instance_count = 1,
                      train_instance_type='ml.c4.xlarge',
                      predictor_type='binary_classifier',
                      output_path=output_path,
                      sagemaker_session=sagemaker_session,
                      epochs = 15)


In [9]:
train_path = output_path + '/train.csv'
train = pd.read_csv(train_path, header=None, names=None, dtype='float32')
train.head()

,0,1,2,3
0,0.0,0.398148,0.000000,0.191781
1,1.0,0.869369,0.449541,0.846491
2,1.0,0.593583,0.081967,0.316062
3,0.0,0.544503,0.000000,0.242574
4,0.0,0.329502,0.000000,0.161172


In [10]:
train_x_np = train.iloc[:,1:].to_numpy()
train_x_np

array([[0.39814815, 0.        , 0.19178082],
       [0.8693694 , 0.44954127, 0.8464912 ],
       [0.59358287, 0.08196721, 0.31606218],
       [0.5445026 , 0.        , 0.24257426],
       [0.32950193, 0.        , 0.16117217],
       [0.59030837, 0.        , 0.30165288],
       [0.75977653, 0.24571429, 0.48430493],
       [0.516129  , 0.        , 0.27083334],
       [0.4408602 , 0.        , 0.22395833],
       [0.9794521 , 0.7887324 , 0.9       ],
       [0.9513889 , 0.5214286 , 0.89403975],
       [0.9764706 , 0.57831323, 0.8232044 ],
       [0.8117647 , 0.28313252, 0.4597701 ],
       [0.44117647, 0.        , 0.30555555],
       [0.4888889 , 0.        , 0.2826087 ],
       [0.81395346, 0.63414633, 0.7888889 ],
       [0.6111111 , 0.        , 0.32467532],
       [1.        , 0.96590906, 1.        ],
       [0.6340206 , 0.00526316, 0.36893204],
       [0.5829384 , 0.08695652, 0.41666666],
       [0.63793105, 0.30701753, 0.48987854],
       [0.42038217, 0.        , 0.21875   ],
       [0.

In [11]:
train_y_np = train.iloc[:,0].to_numpy()
train_y_np

array([0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 0.], dtype=float32)

In [12]:
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

In [13]:
%%time
linear.fit(formatted_train_data)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-14 12:56:50 Starting - Starting the training job...
2020-10-14 12:56:53 Starting - Launching requested ML instances......
2020-10-14 12:57:57 Starting - Preparing the instances for training......
2020-10-14 12:59:20 Downloading - Downloading input data......
2020-10-14 13:00:12 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/14/2020 13:00:16 INFO 139894780659520] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'ini

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.6134925842285156, "sum": 0.6134925842285156, "min": 0.6134925842285156}}, "EndTime": 1602680417.566427, "Dimensions": {"model": 3, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1602680417.566408}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.35053542000906807, "sum": 0.35053542000906807, "min": 0.35053542000906807}}, "EndTime": 1602680417.566492, "Dimensions": {"model": 4, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1602680417.566474}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.34576388767787386, "sum": 0.34576388767787386, "min": 0.34576388767787386}}, "EndTime": 1602680417.566554, "Dimensions": {"model": 5, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear 


2020-10-14 13:00:31 Uploading - Uploading generated training model
2020-10-14 13:00:31 Completed - Training job completed
Training seconds: 71
Billable seconds: 71
CPU times: user 635 ms, sys: 14.5 ms, total: 650 ms
Wall time: 4min 12s


## EXERCÍCIO: Implantar o modelo treinado

Após o treinamento, implante seu modelo para criar um `preditor`. Se você estiver usando um modelo PyTorch, você precisará criar um `PyTorchModel` treinado que aceita o` <model> .model_data` treinado como um parâmetro de entrada e aponta para o arquivo `source_pytorch / predict.py` fornecido como um ponto de entrada.

Para implantar um modelo treinado, você usará `<model> .deploy`, que leva em dois argumentos:
* **initial_instance_count**: O número de instâncias implementadas (1).
* **instance_type**: O tipo de instância SageMaker para implantação.

Observação: se você se deparar com um erro de instância, pode ser porque escolheu o treinamento ou implantação incorreto instance_type. Pode ser útil consultar o código do exercício anterior para ver quais tipos de instâncias usamos.

In [14]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel

# deploy your model to create a predictor
predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!CPU times: user 279 ms, sys: 14.1 ms, total: 293 ms
Wall time: 7min 32s


---
# Avaliando seu modelo

Uma vez que seu modelo é implantado, você pode ver como ele funciona quando aplicado aos nossos dados de teste.

A célula fornecida abaixo lê os dados de teste, presumindo que estejam armazenados localmente em `data_dir` e com o nome` test.csv`. Os rótulos e recursos são extraídos do arquivo `.csv`.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCÍCIO: Determine a precisão do seu modelo

Use seu `preditor` implantado para gerar rótulos de classe previstos para os dados de teste. Compare-os com os rótulos *true*, `test_y`, e calcule a precisão como um valor entre 0 e 1,0 que indica a fração dos dados de teste que seu modelo classificou corretamente. Você pode usar [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) para este cálculo.

**Para ser aprovado neste projeto, seu modelo deve obter pelo menos 90% de precisão de teste.**

In [16]:
predictor.endpoint

'linear-learner-2020-10-14-12-56-50-238'

In [76]:
# example sourced from linear_learner_mnist in sagemaker examples
#specify how to serialize requests and deserialize responses that are specific to the algorithm.
#from sagemaker.predictor import csv_serializer, json_deserializer

#predictor.content_type = 'text/csv'
#predictor.serializer = csv_serializer
#predictor.deserializer = json_deserializer

In [19]:
# First: generate predicted, class labels
test_preds = predictor.predict(test_x.values.astype('float32'))
test_y_preds = np.array([x.label['predicted_label'].float32_tensor.values[0] for x in test_preds])
test_y_preds

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [21]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, test_y_preds, normalize=True)
print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
#print(test_y_preds)
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0.
 0.]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [22]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_y.values, test_y_preds)
cm

array([[ 9,  1],
       [ 0, 15]])

### Pergunta 1: Quantos falsos positivos e falsos negativos seu modelo produziu, se houver? E por que você acha que isso acontece?

O modelo se ajustou muito bem ao problema, assim como desejado. Houve apenas um caso de classificação errada, que foi um falso positivo.

### Pergunta 2: Como você decidiu sobre o tipo de modelo a ser usado?

O classificador LinearLeaner foi escolhido por ser uma das opções sugeridas na prospota do projeto, e por se tratar de uma classificação binária com bom desempenho.

----
## EXERCÍCIO: Limpar recursos

Depois de avaliar seu modelo, **exclua o endpoint do modelo**. Você pode fazer isso com uma chamada para `.delete_endpoint ()`. Você precisa mostrar, neste bloco de notas, que o endpoint foi excluído. Quaisquer outros recursos, você pode excluir do console AWS, e você encontrará mais instruções sobre como limpar todos os seus recursos, abaixo.

In [31]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()

### Excluindo bucket S3

Quando você tiver * completamente * concluído o treinamento e os modelos de teste, também poderá excluir todo o seu bucket S3. Se você fizer isso antes de terminar de treinar seu modelo, terá que recriar seu bucket S3 e carregar seus dados de treinamento novamente.

In [32]:
# deleting bucket, uncomment lines below
 bucket_to_delete = boto3.resource('s3').Bucket(bucket)
 bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'D72FD873236B9E2C',
   'HostId': 'LybVRogpeb0PJ6YVimEYTNViFJnqSLF0sEGXvL9gON2XtfgeKOHMv2nNeVfoD115Yz05rbYnKnQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'LybVRogpeb0PJ6YVimEYTNViFJnqSLF0sEGXvL9gON2XtfgeKOHMv2nNeVfoD115Yz05rbYnKnQ=',
    'x-amz-request-id': 'D72FD873236B9E2C',
    'date': 'Wed, 14 Oct 2020 13:43:41 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism-detection/train.csv'},
   {'Key': 'plagiarism-detection/linear-learner-2020-10-14-12-56-50-238/output/model.tar.gz'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-10-14-12-56-50-089/.amazon.manifest'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-10-14-12-56-50-089/matrix_0.pbr'},
   {'Key': 'plagiarism-detection/test.csv'}]}]

### Excluindo todos os seus modelos e instâncias

Quando você estiver _completamente_ concluído com este projeto e **nunca** quiser revisitar este bloco de notas, você pode escolher excluir todas as instâncias e modelos de bloco de notas SageMaker seguindo [estas instruções](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Antes de excluir esta instância do notebook, recomendo pelo menos baixar uma cópia e salvá-la, localmente.

---
## Outras direções

Existem muitas maneiras de melhorar ou adicionar a este projeto para expandir seu aprendizado ou torná-lo um projeto único para você. Algumas ideias estão listadas abaixo:
* Treine um classificador para prever a *categoria* (1-3) de plágio e não apenas plagiado (1) ou não (0).
* Utilize um conjunto de dados diferente e maior para ver se este modelo pode ser estendido a outros tipos de plágio.
* Use a análise de linguagem ou nível de caractere para encontrar diferentes (e mais) recursos de semelhança.
* Escreva uma função de pipeline completa que aceite um texto-fonte e um arquivo de texto enviado e classifique o texto enviado como plagiado ou não.
* Use o API Gateway e uma função lambda para implantar seu modelo em um aplicativo da web.

Essas são apenas opções para estender seu trabalho. Se você completou todos os exercícios neste bloco de notas, você completou um aplicativo do mundo real e pode prosseguir para enviar seu projeto. Bom trabalho!